# __Regresiones Robustas__

Son tecnicas de regresion que resisten mejor los efectos de valores atipicos en los datos. A diferencia de la regresion lineal estandar, que minimiza el error cuadratico medio (MSE) y puede verse afectada por outliers, las regresiones robustas utilizan funciones de perdida menos sensibles a los outliers.

Por lo general siempre se cuenta con una fase de preprocesamiento en la cual se eliminan o transforman datos y se tratan los outliers. Sin embargo, hay ocasiones donde se requiere tratar estos valores directamente durante la aplicacion del modelo y es alli donde toman relevancia los modelos robustos que facilitan este proceso. 

Debemos entender que los valores atipicos son observaciones que no siguen el patron de las demas observaciones y violan los supuestos en los que se basa MSE lo que compromete el resultado de la regresion lineal.

Las tecnicas de regresion robusta que nos ofrece Scikit - Learn son:

- Huger Regressor.
- RANSAC Regressor.
- TheilSenRegressor.

En lo que sigue veamos como funciona y como se implementa cada una de ellas

## __Huger Regressor__

La regresion Huber modifica la funcion de perdida de la regresion lineal para reducir el impacto de los valores atipicos. Combina las fortalezas del Error Cuadratico Medio (MSE) y del Error Absoluto Medio (MAE), cambiando de MSE a MAE cuando los errores superan un umbral especifico representado por $\delta$

La funcion de perdida de Huber viene representada por:

$$L_{\delta}(r) = \left\{\begin{matrix}
\frac{1}{2}r^2 & si \left | r\right | \leq \delta\\
\delta(\left | r\right | - \frac{1}{2}\delta) &  si \left | r\right | > \delta\\
\end{matrix}\right.$$

Donde,

- $r = y_i - \hat{y_{i}}$, es el error entre la prediccion y el valor real (MSE).

- $\delta$ es un parametro que define cuando el error deja de ser cuadratico (MSE) y pasa a ser lineal (MAE).

Si el error es pequeno ($\left | r\right | \leq \delta$) tratamos el outlier como un dato confiable y lo penalizamos con MSE. En cambio, si el error es grande ($\left | r\right | > \delta$) tratamos el outlier con mayor severidad penalizandolo con MAE